In [1]:
import pandas as pd
import numpy as np
import string
import pickle
from sklearn.metrics import classification_report 
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import corpora, models
import gensim
import nltk
from scipy import spatial

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /home/azmi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/azmi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azmi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/azmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
bugs=pd.read_csv("eclipse_jdt.csv")

In [3]:
bugs.head()

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1518,P1,Debug,NaN,Icons needed for actions (1GI5UXW),JGS (8/8/01 5:20:19 PM); \tWe need enabled; di...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2001-10-18 11:51:14 -0400
1,1519,P3,Debug,NaN,README: Hit count not reset (1GET20Y),JGS (6/4/01 11:43:47 AM); \tSet a breakpoint i...,RESOLVED,WORKSFORME,2.0,2001-10-10 22:14:00 -0400,2001-11-28 13:42:46 -0500
2,1520,P3,Debug,NaN,Use styled text in console (1G9S1YF),DW (2/26/01 5:56:36 PM); \tCould allow users t...,CLOSED,WONTFIX,2.0,2001-10-10 22:14:00 -0400,2002-06-26 11:32:05 -0400
3,1521,P3,Debug,NaN,StringBuffer representation (1GE3BFA),JGS (5/21/01 2:09:10 PM); \tTo be consistent; ...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2002-02-05 10:37:09 -0500
4,1522,P2,Debug,1663.0,Feature: use #toString to display variable val...,DW (9/27/00 11:12:31 AM); \tCurrently; the typ...,RESOLVED,DUPLICATE,2.0,2001-10-10 22:14:00 -0400,2001-10-17 10:17:47 -0400


In [4]:
bugs['Title_Desc_Final'] = bugs['Title'].astype(str) + " " +bugs['Description'].astype(str)
type(bugs['Title_Desc_Final'][4])

str

In [9]:
def remove_punctuation(text):
    no_punct="".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenizer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words=[w for w in text if w not in stopwords.words('english')]
    return words
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text
stemmer = PorterStemmer()

def remove_numeric(text):
    words=[w for w in text if not w.isnumeric()]
    return words

def word_stemmer(text):
    stem_text = [stemmer.stem(i) for i in text]
    return stem_text

In [10]:
bugs_test = bugs[bugs['Resolution']=='DUPLICATE']

In [12]:
bugs['Title_Desc_Final']= bugs['Title_Desc_Final'].apply(lambda x: remove_punctuation(x))
bugs['Title_Desc_Final']= bugs['Title_Desc_Final'].apply(lambda x: tokenizer.tokenize(x.lower()))
bugs['Title_Desc_Final']= bugs['Title_Desc_Final'].apply(lambda x: remove_stopwords(x))
bugs['Title_Desc_Final']= bugs['Title_Desc_Final'].apply(lambda x: remove_numeric(x))
bugs['Title_Desc_Final']= bugs['Title_Desc_Final'].apply(lambda x: word_lemmatizer(x))
#bugs['Title_Final']= bugs['Title_Final'].apply(lambda x: word_stemmer(x))

In [13]:
bugs['Title_Desc_Final'].head(20)

0     [icon, needed, action, 1gi5uxw, jgs, pm, need,...
1     [readme, hit, count, reset, 1get20y, jgs, set,...
2     [use, styled, text, console, 1g9s1yf, dw, pm, ...
3     [stringbuffer, representation, 1ge3bfa, jgs, p...
4     [feature, use, tostring, display, variable, va...
5     [breakpoint, invalid, location, 1g4f8p8, d, pm...
6     [feature, displaying, instruction, pointer, 1g...
7     [feature, lock, monitor, 1g3a7zh, dw, support,...
8     [snippet, evaluation, support, import, 1g47213...
9     [scrapbook, snippet, editor, color, snippet, o...
10    [snippet, start, swt, program, twice, 1g47cme,...
11    [exit, allow, user, continue, execution, targe...
12    [ui, inconsistent, gui, behavior, breakpoints,...
13    [feature, navigate, console, output, resource,...
14    [ui, right, click, run, button, 1g5t50t, dw, p...
15    [variable, snapshot, compare, jgs, pm, involve...
16    [dcr, support, debug, demand, 1g71c81, eg, pm,...
17    [performance, optimization, handle, really

In [14]:
bugs

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Title_Desc_Final
0,1518,P1,Debug,NaN,Icons needed for actions (1GI5UXW),JGS (8/8/01 5:20:19 PM); \tWe need enabled; di...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2001-10-18 11:51:14 -0400,"[icon, needed, action, 1gi5uxw, jgs, pm, need,..."
1,1519,P3,Debug,NaN,README: Hit count not reset (1GET20Y),JGS (6/4/01 11:43:47 AM); \tSet a breakpoint i...,RESOLVED,WORKSFORME,2.0,2001-10-10 22:14:00 -0400,2001-11-28 13:42:46 -0500,"[readme, hit, count, reset, 1get20y, jgs, set,..."
2,1520,P3,Debug,NaN,Use styled text in console (1G9S1YF),DW (2/26/01 5:56:36 PM); \tCould allow users t...,CLOSED,WONTFIX,2.0,2001-10-10 22:14:00 -0400,2002-06-26 11:32:05 -0400,"[use, styled, text, console, 1g9s1yf, dw, pm, ..."
3,1521,P3,Debug,NaN,StringBuffer representation (1GE3BFA),JGS (5/21/01 2:09:10 PM); \tTo be consistent; ...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2002-02-05 10:37:09 -0500,"[stringbuffer, representation, 1ge3bfa, jgs, p..."
4,1522,P2,Debug,1663.0,Feature: use #toString to display variable val...,DW (9/27/00 11:12:31 AM); \tCurrently; the typ...,RESOLVED,DUPLICATE,2.0,2001-10-10 22:14:00 -0400,2001-10-17 10:17:47 -0400,"[feature, use, tostring, display, variable, va..."
...,...,...,...,...,...,...,...,...,...,...,...,...
45291,424739,P3,Core,NaN,During my development i am facing lots of prob...,-- Configuration Details --; Product: Eclipse ...,VERIFIED,WORKSFORME,4.4,2013-12-30 02:05:00 -0500,2014-01-21 04:02:11 -0500,"[development, facing, lot, problem, using, sho..."
45292,424742,P3,Core,NaN,[1.8] NPE in LambdaExpression.isCompatibleWith,package two.test;; ; class TestInlineLambdaArr...,RESOLVED,FIXED,4.4,2013-12-30 04:35:00 -0500,2014-01-02 08:03:23 -0500,"[npe, lambdaexpressioniscompatiblewith, packag..."
45293,424772,P3,Text,NaN,[typing] Correct Indentation for { is wrong wh...,Created attachment 238617; Zipped demo project...,RESOLVED,FIXED,4.4,2013-12-31 03:33:00 -0500,2014-01-14 10:20:33 -0500,"[typing, correct, indentation, wrong, declarat..."
45294,424784,P3,UI,NaN,yes,NaN,CLOSED,INVALID,4.3.1,2013-12-31 21:18:00 -0500,2014-01-01 01:18:07 -0500,"[yes, nan]"


In [15]:
# Convert tokenized document into gensim formated tagged data
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(bugs['Title_Desc_Final'])]
tagged_data

[TaggedDocument(words=['icon', 'needed', 'action', '1gi5uxw', 'jgs', 'pm', 'need', 'enabled', 'disabled', 'hover', 'icon', 'following', 'action', 'copytoclipboardactiondelegate', 'relaunchactiondelegate', 'terminateandremoveactiondelegate', 'dw', 'pm', 'use', 'standard', 'copy', 'icon', 'copy', 'clipboard', 'desktop', 'likely', 'expose', 'dw', 'pm', 'made', 'request', 'relaunch', 'terminate', 'terminate', 'remove'], tags=[0]),
 TaggedDocument(words=['readme', 'hit', 'count', 'reset', '1get20y', 'jgs', 'set', 'breakpoint', 'infiniteloop', 'hit', 'count', 'suspended', 'right', 'click', 'breakpoint', 'breakpoints', 'viewer', 'enable', 'hit', 'count', 'stay', 'previous', 'value', 'reset', 'zero', 'dw', 'reenable', 'breakpoint', 'previous', 'hitcount', 'd', 'change', 'hit', 'count', 'breakpoint', 'normal', 'breakpoint', 'set', 'hit', 'count', 'zero', 'likely', 'readme', 'item', 'dw', 'pm', 'make', 'readme', 'dw', 'added', 'readme'], tags=[1]),
 TaggedDocument(words=['use', 'styled', 'text',

In [21]:
model = Doc2Vec(tagged_data, vector_size=300, window=2, min_count=1, workers=4, epochs = 20)
model.save("test_doc2vec.model")
model= Doc2Vec.load("test_doc2vec.model")
model.wv.vocab

{'icon': <gensim.models.keyedvectors.Vocab at 0x7f2417e132d0>,
 'needed': <gensim.models.keyedvectors.Vocab at 0x7f2417e13450>,
 'action': <gensim.models.keyedvectors.Vocab at 0x7f2417e13b90>,
 '1gi5uxw': <gensim.models.keyedvectors.Vocab at 0x7f2417e13c90>,
 'jgs': <gensim.models.keyedvectors.Vocab at 0x7f2417e13d90>,
 'pm': <gensim.models.keyedvectors.Vocab at 0x7f2417e13e90>,
 'need': <gensim.models.keyedvectors.Vocab at 0x7f2417e13f90>,
 'enabled': <gensim.models.keyedvectors.Vocab at 0x7f2417e13190>,
 'disabled': <gensim.models.keyedvectors.Vocab at 0x7f2417e13410>,
 'hover': <gensim.models.keyedvectors.Vocab at 0x7f2417e135d0>,
 'following': <gensim.models.keyedvectors.Vocab at 0x7f2417e13710>,
 'copytoclipboardactiondelegate': <gensim.models.keyedvectors.Vocab at 0x7f2417e13750>,
 'relaunchactiondelegate': <gensim.models.keyedvectors.Vocab at 0x7f2417e13850>,
 'terminateandremoveactiondelegate': <gensim.models.keyedvectors.Vocab at 0x7f2417e13910>,
 'dw': <gensim.models.keyedvec

In [17]:
def get_duplicate_issue_id(dataset, issue_id):
    if str(dataset[dataset["Issue_id"]==issue_id]['Duplicated_issue'].values[0])=='nan':
        return None
    return dataset[dataset["Issue_id"]==issue_id]['Duplicated_issue'].values[0].astype(int)


def is_issue_exist_in_dataset(dataset,issue_id):
    return len(dataset[dataset["Issue_id"]==issue_id]) != 0

def get_issue_id_with_row( row_number):
    return bugs['Issue_id'][row_number]

def get_issue_title_with_id(dataset, issue_id):
    return dataset[dataset["Issue_id"]==issue_id]['Description'].values[0]

def get_issue_title_final_with_id(dataset, issue_id):
    return dataset[dataset["Issue_id"]==issue_id]['Title_Desc_Final'].values[0]

def get_n_most_similar_docs(dataset, issue_id, n):
    test_doc = get_issue_title_final_with_id(dataset, issue_id)
    similars = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)],topn=len(dataset)-1)
 
    issue_list=[]
    for i in range(0,len(similars)-1):
        if is_component_same(dataset, issue_id, get_issue_id_with_row(similars[i][0])) and issue_id != similars[i][0]:
            issue_list.append(get_issue_id_with_row(similars[i][0]))

        if len(issue_list)==n:
            return issue_list
                
    return issue_list

def get_component_with_id(dataset, issue_id):
    return dataset[dataset["Issue_id"]==issue_id]['Component'].values[0]

def is_component_same(dataset, issue1, issue2):
    return get_component_with_id(dataset, issue1) == get_component_with_id(dataset, issue2)

def duplicate_issue_id_exists_in_similarity_list(duplicate_id, similarity_list):
    return duplicate_id in similarity_list

In [ ]:
count=0
count1=0
for item in bugs_test["Issue_id"]:
    if is_issue_exist_in_dataset(bugs, get_duplicate_issue_id(bugs, item)):
        count1=count1+1
        print("Issue id: " + str(item) + " Duplicate issue id: "+ str(get_duplicate_issue_id(bugs, item)))
        print(get_component_with_id(bugs,get_duplicate_issue_id(bugs, item)))
        if is_component_same(bugs, item, get_duplicate_issue_id(bugs, item)):
            count=count+1

In [ ]:
(100*count)/count1

In [ ]:
count

In [ ]:
count1

In [ ]:
(100*count1)/len(bugs_test)

In [22]:
tp=0
fn=0
for issue in bugs_test['Issue_id']:
    print("Issue: " + str(issue)+" Duplicate Issue: " +str(get_duplicate_issue_id(bugs_test, issue)))
    similar_list=get_n_most_similar_docs(bugs, issue, 20)
    if duplicate_issue_id_exists_in_similarity_list(get_duplicate_issue_id(bugs_test,issue),similar_list):
        tp=tp+1
    else:
        fn=fn+1
    

Issue: 1522 Duplicate Issue: 1663
Issue: 1523 Duplicate Issue: 1555
Issue: 1526 Duplicate Issue: 1633
Issue: 1536 Duplicate Issue: 16036
Issue: 1538 Duplicate Issue: 1605
Issue: 1544 Duplicate Issue: 1630
Issue: 1545 Duplicate Issue: 44986
Issue: 1553 Duplicate Issue: 5346
Issue: 1554 Duplicate Issue: 1552
Issue: 1555 Duplicate Issue: 1568
Issue: 1557 Duplicate Issue: 1630
Issue: 1560 Duplicate Issue: 12018
Issue: 1566 Duplicate Issue: 1532
Issue: 1568 Duplicate Issue: 1570
Issue: 1570 Duplicate Issue: 1619
Issue: 1575 Duplicate Issue: 1524
Issue: 1586 Duplicate Issue: 1552
Issue: 1594 Duplicate Issue: 7552
Issue: 1598 Duplicate Issue: 1585
Issue: 1604 Duplicate Issue: 4383
Issue: 1606 Duplicate Issue: 5346
Issue: 1608 Duplicate Issue: 5163
Issue: 1612 Duplicate Issue: 6185
Issue: 1616 Duplicate Issue: 1546
Issue: 1618 Duplicate Issue: 6509
Issue: 1619 Duplicate Issue: 1667
Issue: 1620 Duplicate Issue: 1546
Issue: 1622 Duplicate Issue: 1524
Issue: 1634 Duplicate Issue: 1585
Issue: 1639

Issue: 6045 Duplicate Issue: 3420
Issue: 6048 Duplicate Issue: 5761
Issue: 6053 Duplicate Issue: 6058
Issue: 6056 Duplicate Issue: 6057
Issue: 6070 Duplicate Issue: 5852
Issue: 6084 Duplicate Issue: 1720
Issue: 6085 Duplicate Issue: 9151
Issue: 6094 Duplicate Issue: 3320
Issue: 6120 Duplicate Issue: 3625
Issue: 6128 Duplicate Issue: 5789
Issue: 6129 Duplicate Issue: 5761
Issue: 6136 Duplicate Issue: 6325
Issue: 6145 Duplicate Issue: 9151
Issue: 6150 Duplicate Issue: 5875
Issue: 6160 Duplicate Issue: 6151
Issue: 6161 Duplicate Issue: 1753
Issue: 6165 Duplicate Issue: 13905
Issue: 6167 Duplicate Issue: 9185
Issue: 6176 Duplicate Issue: 6151
Issue: 6256 Duplicate Issue: 6241
Issue: 6280 Duplicate Issue: 4134
Issue: 6291 Duplicate Issue: 1635
Issue: 6294 Duplicate Issue: 6160
Issue: 6301 Duplicate Issue: 3331
Issue: 6338 Duplicate Issue: 3847
Issue: 6339 Duplicate Issue: 6302
Issue: 6398 Duplicate Issue: 5835
Issue: 6417 Duplicate Issue: 11403
Issue: 6455 Duplicate Issue: 15262
Issue: 6468

Issue: 9726 Duplicate Issue: 9476
Issue: 9766 Duplicate Issue: 9732
Issue: 9785 Duplicate Issue: 7485
Issue: 9843 Duplicate Issue: 13113
Issue: 9853 Duplicate Issue: 7638
Issue: 9880 Duplicate Issue: 1546
Issue: 9888 Duplicate Issue: 9479
Issue: 9890 Duplicate Issue: 13034
Issue: 9897 Duplicate Issue: 18994
Issue: 9925 Duplicate Issue: 22199
Issue: 9939 Duplicate Issue: 9914
Issue: 9949 Duplicate Issue: 5761
Issue: 9951 Duplicate Issue: 12740
Issue: 10029 Duplicate Issue: 6454
Issue: 10041 Duplicate Issue: 6949
Issue: 10045 Duplicate Issue: 11024
Issue: 10048 Duplicate Issue: 9909
Issue: 10051 Duplicate Issue: 9798
Issue: 10097 Duplicate Issue: 5646
Issue: 10129 Duplicate Issue: 7605
Issue: 10137 Duplicate Issue: 9673
Issue: 10151 Duplicate Issue: 10096
Issue: 10166 Duplicate Issue: 9985
Issue: 10172 Duplicate Issue: 9017
Issue: 10202 Duplicate Issue: 10198
Issue: 10219 Duplicate Issue: 8772
Issue: 10228 Duplicate Issue: 5761
Issue: 10237 Duplicate Issue: 8919
Issue: 10238 Duplicate Is

Issue: 13421 Duplicate Issue: 12823
Issue: 13437 Duplicate Issue: 13323
Issue: 13462 Duplicate Issue: 13684
Issue: 13468 Duplicate Issue: 13393
Issue: 13487 Duplicate Issue: 13355
Issue: 13497 Duplicate Issue: 89854
Issue: 13508 Duplicate Issue: 13336
Issue: 13522 Duplicate Issue: 13355
Issue: 13533 Duplicate Issue: 13512
Issue: 13538 Duplicate Issue: 14597
Issue: 13604 Duplicate Issue: 13865
Issue: 13612 Duplicate Issue: 12970
Issue: 13619 Duplicate Issue: 12970
Issue: 13627 Duplicate Issue: 7004
Issue: 13713 Duplicate Issue: 14045
Issue: 13742 Duplicate Issue: 3733
Issue: 13765 Duplicate Issue: 36718
Issue: 13827 Duplicate Issue: 10169
Issue: 13841 Duplicate Issue: 13834
Issue: 13845 Duplicate Issue: 7245
Issue: 13852 Duplicate Issue: 11410
Issue: 13855 Duplicate Issue: 9355
Issue: 13863 Duplicate Issue: 13865
Issue: 13872 Duplicate Issue: 12561
Issue: 13881 Duplicate Issue: 10169
Issue: 13889 Duplicate Issue: 13581
Issue: 13911 Duplicate Issue: 13909
Issue: 13958 Duplicate Issue: 35

Issue: 17723 Duplicate Issue: 19721
Issue: 17726 Duplicate Issue: 17791
Issue: 17727 Duplicate Issue: 17491
Issue: 17732 Duplicate Issue: 19721
Issue: 17750 Duplicate Issue: 12729
Issue: 17797 Duplicate Issue: 16761
Issue: 17798 Duplicate Issue: 16170
Issue: 17804 Duplicate Issue: 17453
Issue: 17824 Duplicate Issue: 19349
Issue: 17827 Duplicate Issue: 16761
Issue: 17856 Duplicate Issue: 14061
Issue: 17866 Duplicate Issue: 15212
Issue: 17888 Duplicate Issue: 16534
Issue: 17949 Duplicate Issue: 17512
Issue: 17950 Duplicate Issue: 16638
Issue: 17953 Duplicate Issue: 16761
Issue: 18011 Duplicate Issue: 15170
Issue: 18012 Duplicate Issue: 18011
Issue: 18013 Duplicate Issue: 17095
Issue: 18014 Duplicate Issue: 18079
Issue: 18022 Duplicate Issue: 1609
Issue: 18070 Duplicate Issue: 16761
Issue: 18084 Duplicate Issue: 11077
Issue: 18098 Duplicate Issue: 19581
Issue: 18107 Duplicate Issue: 17868
Issue: 18113 Duplicate Issue: 15212
Issue: 18171 Duplicate Issue: 14555
Issue: 18203 Duplicate Issue:

...

Issue: 246637 Duplicate Issue: 243692
Issue: 246659 Duplicate Issue: 243692
Issue: 246685 Duplicate Issue: 243692
Issue: 246692 Duplicate Issue: 244716
Issue: 246715 Duplicate Issue: 246712
Issue: 246728 Duplicate Issue: 266582
Issue: 246816 Duplicate Issue: 32340
Issue: 246849 Duplicate Issue: 163554
Issue: 247178 Duplicate Issue: 243692
Issue: 247809 Duplicate Issue: 164449
Issue: 247848 Duplicate Issue: 243101
Issue: 247905 Duplicate Issue: 83651
Issue: 248080 Duplicate Issue: 244716
Issue: 248246 Duplicate Issue: 274898
Issue: 248303 Duplicate Issue: 249456
Issue: 248329 Duplicate Issue: 243179
Issue: 248537 Duplicate Issue: 170431
Issue: 248560 Duplicate Issue: 243692
Issue: 248628 Duplicate Issue: 239647
Issue: 248650 Duplicate Issue: 275308
Issue: 248781 Duplicate Issue: 245973
Issue: 249241 Duplicate Issue: 277574
Issue: 249425 Duplicate Issue: 21402
Issue: 249677 Duplicate Issue: 241155
Issue: 249726 Duplicate Issue: 249456
Issue: 249732 Duplicate Issue: 283950
Issue: 249756 D

Issue: 275801 Duplicate Issue: 255381
Issue: 275876 Duplicate Issue: 269182
Issue: 275918 Duplicate Issue: 194358
Issue: 276021 Duplicate Issue: 269182
Issue: 276847 Duplicate Issue: 183880
Issue: 277301 Duplicate Issue: 277812
Issue: 277302 Duplicate Issue: 277812
Issue: 277305 Duplicate Issue: 280668
Issue: 277432 Duplicate Issue: 38185
Issue: 277895 Duplicate Issue: 237090
Issue: 277897 Duplicate Issue: 236818
Issue: 277952 Duplicate Issue: 267178
Issue: 278146 Duplicate Issue: 341232
Issue: 278236 Duplicate Issue: 277697
Issue: 278749 Duplicate Issue: 261198
Issue: 278808 Duplicate Issue: 195834
Issue: 279402 Duplicate Issue: 248954
Issue: 279466 Duplicate Issue: 234769
Issue: 280054 Duplicate Issue: 277643
Issue: 280220 Duplicate Issue: 267616
Issue: 280298 Duplicate Issue: 266510
Issue: 280410 Duplicate Issue: 278745
Issue: 280550 Duplicate Issue: 250425
Issue: 280563 Duplicate Issue: 250425
Issue: 280809 Duplicate Issue: 277643
Issue: 280819 Duplicate Issue: 65876
Issue: 280951 

Issue: 313086 Duplicate Issue: 311863
Issue: 313212 Duplicate Issue: 251200
Issue: 313308 Duplicate Issue: 312867
Issue: 313737 Duplicate Issue: 58205
Issue: 313824 Duplicate Issue: 181788
Issue: 314073 Duplicate Issue: 139197
Issue: 314596 Duplicate Issue: 172256
Issue: 314931 Duplicate Issue: 313972
Issue: 315491 Duplicate Issue: 112433
Issue: 316090 Duplicate Issue: 289247
Issue: 316678 Duplicate Issue: 257313
Issue: 316782 Duplicate Issue: 189459
Issue: 317012 Duplicate Issue: 289247
Issue: 317105 Duplicate Issue: 283503
Issue: 317313 Duplicate Issue: 172256
Issue: 317367 Duplicate Issue: 298936
Issue: 317576 Duplicate Issue: 123836
Issue: 317762 Duplicate Issue: 317719
Issue: 317784 Duplicate Issue: 308000
Issue: 317829 Duplicate Issue: 321926
Issue: 317836 Duplicate Issue: 315732
Issue: 317893 Duplicate Issue: 309334
Issue: 318180 Duplicate Issue: 280801
Issue: 318243 Duplicate Issue: 313530
Issue: 318247 Duplicate Issue: 317719
Issue: 318276 Duplicate Issue: 168565
Issue: 318298

Issue: 355773 Duplicate Issue: 304176
Issue: 356018 Duplicate Issue: 353535
Issue: 356282 Duplicate Issue: 301990
Issue: 356303 Duplicate Issue: 342332
Issue: 356417 Duplicate Issue: 305172
Issue: 356678 Duplicate Issue: 356677
Issue: 356679 Duplicate Issue: 356677
Issue: 356949 Duplicate Issue: 352990
Issue: 358045 Duplicate Issue: 357918
Issue: 359569 Duplicate Issue: 311212
Issue: 360523 Duplicate Issue: 251824
Issue: 360600 Duplicate Issue: 359646
Issue: 361115 Duplicate Issue: 346175
Issue: 361181 Duplicate Issue: 348024
Issue: 362102 Duplicate Issue: 361916
Issue: 362154 Duplicate Issue: 212262
Issue: 362775 Duplicate Issue: 393192
Issue: 362969 Duplicate Issue: 334622
Issue: 363512 Duplicate Issue: 348507
Issue: 363556 Duplicate Issue: 359173
Issue: 363724 Duplicate Issue: 314556
Issue: 364008 Duplicate Issue: 361053
Issue: 364485 Duplicate Issue: 152568
Issue: 364862 Duplicate Issue: 320487
Issue: 365015 Duplicate Issue: 106228
Issue: 365455 Duplicate Issue: 346175
Issue: 36578

Issue: 415587 Duplicate Issue: 382069
Issue: 415600 Duplicate Issue: 410325
Issue: 415948 Duplicate Issue: 415803
Issue: 415993 Duplicate Issue: 329294
Issue: 416180 Duplicate Issue: 416174
Issue: 416766 Duplicate Issue: 415276
Issue: 416853 Duplicate Issue: 412076
Issue: 416917 Duplicate Issue: 410325
Issue: 417662 Duplicate Issue: 414038
Issue: 417757 Duplicate Issue: 417758
Issue: 417795 Duplicate Issue: 403749
Issue: 417796 Duplicate Issue: 403749
Issue: 418232 Duplicate Issue: 412076
Issue: 418233 Duplicate Issue: 415269
Issue: 418273 Duplicate Issue: 416267
Issue: 418726 Duplicate Issue: 395348
Issue: 418807 Duplicate Issue: 419048
Issue: 418827 Duplicate Issue: 416267
Issue: 418843 Duplicate Issue: 416267
Issue: 419121 Duplicate Issue: 416267
Issue: 419152 Duplicate Issue: 415803
Issue: 419183 Duplicate Issue: 248059
Issue: 419506 Duplicate Issue: 416911
Issue: 419578 Duplicate Issue: 416267
Issue: 419767 Duplicate Issue: 377883
Issue: 419821 Duplicate Issue: 419331
Issue: 41986

In [23]:
print("True Positive: " + str(tp))
print("False Negative: " + str(fn))

True Positive: 1295
False Negative: 6393


In [24]:
print("Recall: " +str(tp/(tp+fn)))

Recall: 0.1684443288241415


# FIREFOX
##### Vector size:200 Epoch:20 Recall: 0.07477522756463953 (True Positive: 2678 False Negative: 33136)
##### Vector size:300 Epoch:20 Recall: 0.07689730273077568 (True Positive: 2754 False Negative: 33060)
##### Vector size:300 Epoch:20 Recall: 0.08661417322834646 (True Positive: 3102 False Negative: 32712) (Component filtreli)
##### Vector size:300 Epoch:300 Recall: 0.07566873289774949 (True Positive: 2710 False Negative: 33104)
##### Vector size:400 Epoch:10 Recall: 0.06704082202490647 (True Positive: 2401 False Negative: 33413)
##### Vector size:400 Epoch:20 Recall: 0.07530574635617357 (True Positive: 2697 False Negative: 33117)

# THUNDERBIRD
##### Vector size:300 Epoch:20 Recall: 0.08383329333653308 (True Positive: 1048 False Negative: 11453) (Component filtreli)

# ECLIPSE_JDT
##### Vector size:300 Epoch:20 Recall: 0.1650624349635796 (True Positive: 1269 False Negative: 6419) (Component filtreli)
##### Vector size:300 Epoch:40 Recall: 0.20096253902185224 (True Positive: 1545 False Negative: 6143) (Component filtreli)
##### Vector size:300 Epoch:100 Recall: 0.2144901144640999 (True Positive: 1649 False Negative: 6039) (Component filtreli)

In [ ]:
import tensorflow as tf